In [3]:
!python -m pip install pyTelegramBotAPI
!pip install selenium
!pip install bs4

  Using cached selenium-3.141.0-py2.py3-none-any.whl (904 kB)
  Using cached bs4-0.0.1-py3-none-any.whl


In [20]:
import telebot  # pip install pyTelegramBotAPI
import requests
from bs4 import BeautifulSoup  # pip install bs4
import selenium
from urllib import parse
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
TOKEN = '1795391045:AAFsmAKo_ResztkMR96grDLq-Ftyf1luPmY'
bot = telebot.TeleBot(TOKEN)

keyword = input('')
@bot.message_handler(commands = [keyword]) # 음식 입력하면 실행
def search_result(message):

    driver = webdriver.Chrome('chromedriver.exe')

    driver.implicitly_wait(10)
    driver.get(f'https://www.diningcode.com/isearch.php?query={keyword}') # 김치찌개 맛집 검색 이동

    location = driver.find_element(By.XPATH,'//*[@id="contents"]/div[1]/div/img')

    action_loc = webdriver.ActionChains(driver)
    action_loc.move_to_element(location)
    action_loc.click() # 내 위치 업데이트
    action_loc.perform() #action함수 실행

    shop = driver.find_element(By.XPATH,'//*[@id="div_normal_nearby"]/ul/li[1]/a/span[1]')
    score = driver.find_element(By.XPATH,'//*[@id="div_normal_nearby"]/ul/li[1]/p[3]')
    voice1 = f'내 주변 {keyword} 맛집검색결과, 맛집 점수 {score.text}. 상위 1순위 {shop.text}입니다. 이동경로를 탐색합니다'


    destination = driver.find_element(By.XPATH,'//*[@id="div_normal_nearby"]/ul/li[1]/a/span[3]')
    dt = destination.text # 목적지, 맛집주소
    # print(dt)


    driver.get('https://map.kakao.com/') # 맛집 길찾기 시작
    time.sleep(3)
    direction = driver.find_element(By.XPATH,'//*[@id="search.tab2"]/a')

    box = driver.find_element(By.XPATH,'/html/body/div[10]/div/div/img')
    action_box = webdriver.ActionChains(driver)
    action_box.move_to_element(box)
    action_box.click()
    action_box.perform()

    direction.click()
    startbox = driver.find_element(By.XPATH,'//*[@id="info.route.searchBox.list"]/div[1]/div[2]/input')
    startpoint = driver.find_element(By.XPATH,'//*[@id="info.route.waypointSuggest.input0"]')
    ds = '서울 강동구 양재대로 1377 (성내동)' # 대신it학원 주소

    action_start = webdriver.ActionChains(driver)
    action_start.move_to_element(startpoint)
    action_start.click()
    action_start.perform()

    startpoint.send_keys(ds, Keys.ENTER) # 출발지입력
    time.sleep(3)

    des = driver.find_element(By.XPATH,'//*[@id="info.route.waypointSuggest.input1"]')
    # des.click()
    des.send_keys(dt, Keys.ENTER) # 목적지입력
    time.sleep(3)
    car = driver.find_element(By.XPATH, '//*[@id="cartab"]')
    car.click()


    car_time = driver.find_element(By.XPATH, '//*[@id="info.flagsearch"]/div[6]/ul/li/div[1]/div/div[1]/p/span[1]')

    car_distance = driver.find_element(By.XPATH, '//*[@id="info.flagsearch"]/div[6]/ul/li/div[1]/div/div[1]/p/span[2]')
    car_info = driver.find_element(By.XPATH, '//*[@id="info.flagsearch"]/div[6]/ul/li/div[1]/div/div[1]/div')
    voice2 =f'자차로 이동시 예상소요시간 {car_time.text}, 거리 {car_distance.text}, 예상 {car_info.text}입니다. 안전운전하세요'



    public_tr = driver.find_element(By.XPATH, '//*[@id="transittab"]')
    public_tr.click()
    pub_time = driver.find_element(By.XPATH, '//*[@id="info.flagsearch"]/div[5]/ul/li[1]/div[1]/span[1]')
    pub_info = driver.find_element(By.XPATH,'//*[@id="info.flagsearch"]/div[5]/ul/li[1]/div[1]/span[2]')
    voice3 = f'대중교통 이용시 예상소요시간 {pub_time.text}, {pub_info.text}. 교통카드를 챙겨주세요.'

    work = driver.find_element(By.XPATH, '//*[@id="walktab"]')
    work.click()
    work_fast_time = driver.find_element(By.XPATH,'//*[@id="info.walkRoute"]/div[1]/ul/li[2]/div[1]/div/p/span[1]')
    work_fast_dis = driver.find_element(By.XPATH,'//*[@id="info.walkRoute"]/div[1]/ul/li[2]/div[1]/div/p/span[2]')
    work_slow_time = driver.find_element(By.XPATH,'//*[@id="info.walkRoute"]/div[1]/ul/li[3]/div[1]/div/p/span[1]')
    work_slow_dis = driver.find_element(By.XPATH,'//*[@id="info.walkRoute"]/div[1]/ul/li[3]/div[1]/div/p/span[2]')
    voice4 = f'도보 이용시 최단거리는 {work_fast_time.text}, {work_fast_dis.text}, 편안한 길은 {work_slow_time.text}, {work_slow_dis.text}'

    bicycle = driver.find_element(By.XPATH, '//*[@id="biketab"]')
    bicycle.click()
    bic_road_time = driver.find_element(By.XPATH,'//*[@id="info.bikeRoute"]/div[1]/ul/li[1]/div[1]/div[1]/p/span[1]')
    bic_road_dis = driver.find_element(By.XPATH,'//*[@id="info.bikeRoute"]/div[1]/ul/li[1]/div[1]/div[1]/p/span[2]')
    bic_roadinfo = driver.find_element(By.XPATH,'//*[@id="info.bikeRoute"]/div[1]/ul/li[1]/div[1]/div[1]/div')
    voice5 = f'자전거 도로 이용시 예상 소요시간{bic_road_time.text}, 거리 {bic_road_dis.text}, {bic_roadinfo.text}'
    
    bot.reply_to(message,voice1)
    bot.reply_to(message,voice2)
    bot.reply_to(message,voice3)
    bot.reply_to(message,voice4)
    bot.reply_to(message,voice5)
    
@bot.message_handler(commands = ['오늘날씨'])
def weather(message):
    driver = webdriver.Chrome('chromedriver.exe')
    driver.implicitly_wait(10)
    driver.get('https://www.google.com/search?q=오늘날씨')
    temp = driver.find_element(By.XPATH,'//*[@id="wob_tm"]') # 온도
    cloud = driver.find_element(By.XPATH,'//*[@id="wob_tci]').get('alt') # 비 여부
    dailycross = 
    
    
    bot.reply_to(message,)
    

bot.polling()

김치찌개


WebDriverException: Message: chrome not reachable
  (Session info: chrome=90.0.4430.212)


In [26]:
driver = webdriver.Chrome('chromedriver.exe')
driver.implicitly_wait(10)
driver.get('https://www.google.com/search?q=오늘날씨')

dc = driver.find_element(By.XPATH,'//*[@id="wob_gsvg"]/text[2]')
print(dc.text)

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="wob_gsvg"]/text[2]"}
  (Session info: chrome=90.0.4430.212)
